In [1]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

In [2]:
test =  pd.read_csv('data/test_full.csv', parse_dates = ['fecha'])
train = pd.read_csv('data/train_full.csv',parse_dates = ['fecha'])

In [3]:
train['habitaciones']=train['habitaciones'].astype(np.int8)
test['habitaciones']=test['habitaciones'].astype(np.int8)

train['metroscubiertos']=train['metroscubiertos'].astype(np.int16)
test['metroscubiertos']=test['metroscubiertos'].astype(np.int16)

train['metrostotales']=train['metrostotales'].astype(np.int16)
test['metrostotales']=test['metrostotales'].astype(np.int16)

train['antiguedad']=train['antiguedad'].astype(np.uint8)
test['antiguedad']=test['antiguedad'].astype(np.uint8)

train['garages']=train['garages'].astype(np.int8)
test['garages']=test['garages'].astype(np.int8)

train['banos']=train['banos'].astype(np.int8)
test['banos']=test['banos'].astype(np.int8)

train['gimnasio']=train['gimnasio'].astype(np.int8)
train['usosmultiples']=train['usosmultiples'].astype(np.int8)
train['piscina']=train['piscina'].astype(np.int8)
train['escuelascercanas']=train['escuelascercanas'].astype(np.int8)
train['centroscomercialescercanos']=train['centroscomercialescercanos'].astype(np.int8)
test['gimnasio']=test['gimnasio'].astype(np.int8)
test['usosmultiples']=test['usosmultiples'].astype(np.int8)
test['piscina']=test['piscina'].astype(np.int8)
test['escuelascercanas']=test['escuelascercanas'].astype(np.int8)
test['centroscomercialescercanos']=test['centroscomercialescercanos'].astype(np.int8)

In [4]:
#Algunas columnas tienen nulls que no se como encararlos. Por ahora los relleno con 0
train=train.fillna(0)
test=test.fillna(0)

# Ensambles

In [2]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

In [ ]:
def crear_submit(model, x_train, y_train, x_test, nombre):
    model.fit(x_train, y_train)
    result = model.predict(x_test)
    for i in range(len(result)):
      result[i] = round(result[i],-3)
    final = test.loc[:, ['id']]
    final['target'] = result
    #Ruta desde Colab
    #ruta = "../content/drive/My Drive/data/submit"+ nombre +".csv"
    #Ruta local
    ruta = "data/submit_"+ nombre +".csv"
    final.to_csv(ruta,index=False)
    return final

In [3]:
train = pd.read_csv("data/features_train_0.csv")
test = pd.read_csv("data/features_test_0.csv")
precios =  pd.read_csv("data/precios.csv")

In [4]:
def mc_mt(x):
    if x["metrostotales"]<x["metroscubiertos"]:
        x["metrostotales"] = x["metroscubiertos"]
    return x

In [5]:
train = train.transform(mc_mt ,1)
test = test.transform(mc_mt ,1)

In [5]:
q=0
for x in train.index:
    if train.loc[x]["metrostotales"]<train.loc[x]["metroscubiertos"]:
        q+=1
q

0

In [6]:
q=0
for x in test.index:
    if test.loc[x]["metrostotales"]<test.loc[x]["metroscubiertos"]:
        q+=1
q

0

In [8]:
from sklearn.ensemble  import RandomForestRegressor,  VotingRegressor, AdaBoostRegressor
import xgboost as xgb

In [9]:
xgb_model = xgb.XGBRegressor(subsample = 0.9, reg_alpha = 1, n_estimators = 200, min_child_weight = 1,\
                             max_features = 43, max_depth = 9, learning_rate = 0.25, gamma = 0.0, \
                             colsample_bytree = 0.9)
rf_model = RandomForestRegressor(random_state = 14, n_estimators = 200, min_samples_split = 5,\
                                 min_samples_leaf = 2, max_features = 51, max_depth = 40, oob_score= True)
abr_model = AdaBoostRegressor(base_estimator=xgb_model, random_state = 22, n_estimators = 11, loss = 'linear', learning_rate = 0.4)
vr_model = VotingRegressor(estimators = [('XGBoost', xgb_model), ('RandomForest', rf_model), ('AdaBoost', abr_model)], weights = [2,1,3])

In [10]:
datetime.now()

datetime.datetime(2019, 11, 30, 22, 51, 47, 865800)

In [11]:
vr_model.fit(train, precios["precios"])

C:\Users\Fernando\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\Fernando\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[22:51:49] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:34:22] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:49:41] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[00:05:09] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[00:20:27] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[00:35:45] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[00:51:05] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:06:31] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:21:53] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

VotingRegressor(estimators=[('XGBoost',
                             XGBRegressor(base_score=0.5, booster='gbtree',
                                          colsample_bylevel=1,
                                          colsample_bynode=1,
                                          colsample_bytree=0.9, gamma=0.0,
                                          importance_type='gain',
                                          learning_rate=0.25, max_delta_step=0,
                                          max_depth=9, max_features=43,
                                          min_child_weight=1, missing=None,
                                          n_estimators=200, n_jobs=1,
                                          nthread=None, objective='reg:linear',
                                          random_state=0,...
                                                                           learning_rate=0.25,
                                                                           max_delta

In [12]:
p = vr_model.predict(test)

In [13]:
datetime.now()

datetime.datetime(2019, 12, 1, 2, 24, 20, 138800)

In [14]:
p

array([6069062.2096911 , 1086965.29300595, 1998808.99355736, ...,
       1012259.17166171, 1630513.73666082, 2176046.26309073])

In [15]:
ids = pd.read_csv("data/test.csv")["id"]

In [16]:
ids

0          4941
1         51775
2        115253
3        299321
4        173570
          ...  
59995     75094
59996    171847
59997    138313
59998    271268
59999     72612
Name: id, Length: 60000, dtype: int64

In [17]:
with open("data/vot_mt_modif.csv","w") as f:
    f.write("id,target\n")
    for i,x in enumerate(p):
        f.write("{},{}\n".format(ids[i],x))

In [ ]:
#496000 en kaggle

# Ahora pruebo un lasso sobre xgb y rf

In [1]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

In [2]:
train = pd.read_csv("data/features_train_0.csv")
test = pd.read_csv("data/features_test_0.csv")
precios =  pd.read_csv("data/precios.csv")

In [3]:
from sklearn.ensemble  import RandomForestRegressor
import xgboost as xgb

In [4]:
xgb_model = xgb.XGBRegressor(subsample = 0.9, reg_alpha = 1, n_estimators = 200, min_child_weight = 1,\
                             max_features = 43, max_depth = 9, learning_rate = 0.25, gamma = 0.0, \
                             colsample_bytree = 0.9)
rf_model = RandomForestRegressor(random_state = 14, n_estimators = 200, min_samples_split = 5,\
                                 min_samples_leaf = 2, max_features = 51, max_depth = 40, oob_score= True)
#abr_model = AdaBoostRegressor(base_estimator=xgb_model, random_state = 22, n_estimators = 11, loss = 'linear', learning_rate = 0.4)
#vr_model = VotingRegressor(estimators = [('XGBoost', xgb_model), ('RandomForest', rf_model), ('AdaBoost', abr_model)], weights = [2,1,3])

In [5]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split

In [6]:
l_model = Lasso()

In [7]:
x_train, x_test, y_train, y_test = train_test_split(train, precios["precios"], test_size=0.1)

In [8]:
xgb_model.fit(x_train, y_train)

C:\Users\Fernando\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[13:49:24] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.9, gamma=0.0,
             importance_type='gain', learning_rate=0.25, max_delta_step=0,
             max_depth=9, max_features=43, min_child_weight=1, missing=None,
             n_estimators=200, n_jobs=1, nthread=None, objective='reg:linear',
             random_state=0, reg_alpha=1, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=0.9, verbosity=1)

In [9]:
rf_model.fit(x_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=40,
                      max_features=51, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=2, min_samples_split=5,
                      min_weight_fraction_leaf=0.0, n_estimators=200,
                      n_jobs=None, oob_score=True, random_state=14, verbose=0,
                      warm_start=False)

In [10]:
pred_xgb = xgb_model.predict(x_test)

In [11]:
pred_rf = rf_model.predict(x_test)

In [12]:
preds = pd.DataFrame()
preds["xgb"] = pred_xgb
preds["rf"] = pred_rf
preds

,xgb,rf
0,1745032.125,1.781233e+06
1,2028298.250,1.946545e+06
2,6252803.000,6.598651e+06
3,2787450.000,3.348406e+06
4,4411388.500,4.417464e+06
...,...,...
23995,1713867.250,1.944648e+06
23996,4880803.000,3.484906e+06
23997,871503.250,1.016736e+06
23998,799901.875,7.313292e+05


In [13]:
l_model.fit(preds, y_test)

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [14]:
coef = l_model.coef_
coef

array([0.55839568, 0.47143515])

In [15]:
indep = l_model.intercept_
indep

-87356.16418385459

In [16]:
params = l_model.get_params()
params

{'alpha': 1.0,
 'copy_X': True,
 'fit_intercept': True,
 'max_iter': 1000,
 'normalize': False,
 'positive': False,
 'precompute': False,
 'random_state': None,
 'selection': 'cyclic',
 'tol': 0.0001,
 'warm_start': False}

In [17]:
xgb_model.fit(train, precios["precios"])

C:\Users\Fernando\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\Fernando\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[14:48:45] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.9, gamma=0.0,
             importance_type='gain', learning_rate=0.25, max_delta_step=0,
             max_depth=9, max_features=43, min_child_weight=1, missing=None,
             n_estimators=200, n_jobs=1, nthread=None, objective='reg:linear',
             random_state=0, reg_alpha=1, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=0.9, verbosity=1)

In [18]:
rf_model.fit(train, precios["precios"])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=40,
                      max_features=51, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=2, min_samples_split=5,
                      min_weight_fraction_leaf=0.0, n_estimators=200,
                      n_jobs=None, oob_score=True, random_state=14, verbose=0,
                      warm_start=False)

In [19]:
pred_f_xgb = xgb_model.predict(test)

In [20]:
pred_f_rf = rf_model.predict(test)

In [21]:
preds_f = pd.DataFrame()
preds_f["xgb"] = pred_f_xgb
preds_f["rf"] = pred_f_rf

In [22]:
submit = l_model.predict(preds_f)

In [23]:
datetime.now()

datetime.datetime(2019, 12, 1, 15, 29, 3, 761000)

In [24]:
submit

array([6109030.67392606, 1086377.65912174, 1971786.02038476, ...,
        946673.1882374 , 1667166.78224986, 2124659.71603971])

In [25]:
ids = pd.read_csv("data/test.csv")["id"]

In [26]:
with open("data/l_xgb_rf.csv","w") as f:
    f.write("id,target\n")
    for i,x in enumerate(submit):
        f.write("{},{}\n".format(ids[i],x))

# Ahora uso más iteraciones para el lasso y fitteo una sola vez 

In [1]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

In [2]:
train = pd.read_csv("data/features_train_0.csv")
test = pd.read_csv("data/features_test_0.csv")
precios =  pd.read_csv("data/precios.csv")

In [3]:
from sklearn.ensemble  import RandomForestRegressor
import xgboost as xgb

In [4]:
xgb_model = xgb.XGBRegressor(subsample = 0.9, reg_alpha = 1, n_estimators = 200, min_child_weight = 1,\
                             max_features = 43, max_depth = 9, learning_rate = 0.25, gamma = 0.0, \
                             colsample_bytree = 0.9)
rf_model = RandomForestRegressor(random_state = 14, n_estimators = 200, min_samples_split = 5,\
                                 min_samples_leaf = 2, max_features = 51, max_depth = 40, oob_score= True)
#abr_model = AdaBoostRegressor(base_estimator=xgb_model, random_state = 22, n_estimators = 11, loss = 'linear', learning_rate = 0.4)
#vr_model = VotingRegressor(estimators = [('XGBoost', xgb_model), ('RandomForest', rf_model), ('AdaBoost', abr_model)], weights = [2,1,3])

In [5]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split

In [6]:
l_model = Lasso(max_iter=10000)

In [7]:
x_train, x_test, y_train, y_test = train_test_split(train, precios["precios"], test_size=0.15)

In [8]:
xgb_model.fit(x_train, y_train)

C:\Users\Fernando\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[16:24:05] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.9, gamma=0.0,
             importance_type='gain', learning_rate=0.25, max_delta_step=0,
             max_depth=9, max_features=43, min_child_weight=1, missing=None,
             n_estimators=200, n_jobs=1, nthread=None, objective='reg:linear',
             random_state=0, reg_alpha=1, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=0.9, verbosity=1)

In [9]:
rf_model.fit(x_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=40,
                      max_features=51, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=2, min_samples_split=5,
                      min_weight_fraction_leaf=0.0, n_estimators=200,
                      n_jobs=None, oob_score=True, random_state=14, verbose=0,
                      warm_start=False)

In [10]:
pred_xgb = xgb_model.predict(x_test)

In [11]:
pred_rf = rf_model.predict(x_test)

In [12]:
preds = pd.DataFrame()
preds["xgb"] = pred_xgb
preds["rf"] = pred_rf
preds

,xgb,rf
0,5.068564e+06,5.677151e+06
1,1.824251e+06,1.974530e+06
2,1.823762e+06,1.820119e+06
3,2.577535e+06,2.170490e+06
4,7.548565e+06,8.032749e+06
...,...,...
35995,1.621554e+06,1.577502e+06
35996,7.099494e+05,7.083673e+05
35997,9.822709e+05,1.059279e+06
35998,4.893732e+05,3.601019e+05


In [13]:
l_model.fit(preds, y_test)

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=10000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [14]:
coef = l_model.coef_
coef

array([0.52129371, 0.50896599])

In [15]:
indep = l_model.intercept_
indep

-83524.63384452416

In [16]:
params = l_model.get_params()
params

{'alpha': 1.0,
 'copy_X': True,
 'fit_intercept': True,
 'max_iter': 10000,
 'normalize': False,
 'positive': False,
 'precompute': False,
 'random_state': None,
 'selection': 'cyclic',
 'tol': 0.0001,
 'warm_start': False}

In [17]:
pred_f_xgb = xgb_model.predict(test)

In [18]:
pred_f_rf = rf_model.predict(test)

In [19]:
preds_f = pd.DataFrame()
preds_f["xgb"] = pred_f_xgb
preds_f["rf"] = pred_f_rf

In [20]:
submit = l_model.predict(preds_f)

In [21]:
submit

array([6329763.88442235,  883938.07093093, 2070588.67210209, ...,
        895495.25822224, 1561822.70729777, 2078637.18647571])

In [22]:
ids = pd.read_csv("data/test.csv")["id"]

In [23]:
with open("data/l_xgb_rf_2.csv","w") as f:
    f.write("id,target\n")
    for i,x in enumerate(submit):
        f.write("{},{}\n".format(ids[i],x))